In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

person_path='/home/hjj/Desktop/'

train_path=person_path+'GcN-sur/train/'

test_path=person_path+'GcN-sur/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
#input of D
global A
A =12.8282906600000004**2

In [3]:
def get_slab_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy_slab=round(np.float64(float(energy_string)),5)
    return energy_slab
def get_bluk_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR_bulk | tail -1').read().split(' ')[-2]
    energy_bluk=round(np.float64(float(energy_string)),5)
    return energy_bluk


def get_cpx222_surface_energy(folder):
    slab=get_slab_energy(folder)
    bluk=get_bluk_energy(folder)
    
    global A
    sur_e= (slab-bluk)/(2*A)
    return sur_e

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
'''
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom
'''

'\ndef get_energy_per_atom(energy):\n    energy_per_atom=energy/atoms_num\n    return energy_per_atom\n'

In [4]:
global extend_num, move_num

In [5]:
extend_num=10

In [6]:
move_num=get_cpx222_surface_energy(train_path+'00000')
print(move_num)

0.13767811697355392


In [7]:
base=mg.Structure.from_file(train_path+'00000/POSCAR')
base_xrd =patt_xrd.get_pattern(base)
base_x=[]
base_y=[]
for i in range(len(base_xrd.y)):
    if base_xrd.y[i] >0.0001 and base_xrd.y[i] < 1:
        base_y.append(base_xrd.y[i])
        base_x.append(base_xrd.x[i])
base_x=base_x[:28]
base_y=base_y[:28]

/home/hjj/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pymatgen/io/cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


In [ ]:
get_xrdmat_slab(tomgStructure(train_path+'00309'))

In [ ]:
(get_xrdmat_slab(tomgStructure(train_path+'00309'))-get_xrdmat_slab(tomgStructure(train_path+'00000')))

In [8]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/POSCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###

def get_xrdmat_slab(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)

    i_column = 28
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i] >0.0001 and xrd_data4.y[i] < 1:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    #xrd_x=xrd_x-base_x
    xrd_y=1000*(xrd_y-base_y)
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=(np.arctan(xrd_y))/180*np.pi
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4

###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat_slab(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [9]:
class GCNNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GCNNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

In [10]:
G1= GCNNet()
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.1)
#mat_Gl=[]    
#pre_gg=[]

In [11]:
trainfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        trainfile.append(ieach)
i=0

In [ ]:
get_cpx222_surface_energy(trainfile[0])

In [12]:

loss_set=[]
for path_ in trainfile:
    E_Gibbs=[]
    try:
        total_energy=get_cpx222_surface_energy(path_)
        tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
    except:
        print(path_)
            
    E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    g_in=[]
           
     
    try:
        tomgS=tomgStructure(path_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)    
    
    G1_loss=torch.abs(torch.mean(Gout-float(tt_energy)))
    

    opt_G1.zero_grad()
    G1_loss.backward()
    opt_G1.step()
    
    i += 1
    loss_set.append(G1_loss)
    print(i,": ",G1_loss)

1 :  tensor(0.08480260, grad_fn=<AbsBackward>)
2 :  tensor(2084.58366634, grad_fn=<AbsBackward>)
3 :  tensor(50.85578628, grad_fn=<AbsBackward>)
4 :  tensor(0.06869130, grad_fn=<AbsBackward>)
5 :  tensor(0.00521490, grad_fn=<AbsBackward>)
6 :  tensor(0.05881272, grad_fn=<AbsBackward>)
7 :  tensor(0.05146252, grad_fn=<AbsBackward>)
8 :  tensor(0.00992214, grad_fn=<AbsBackward>)
9 :  tensor(0.06115112, grad_fn=<AbsBackward>)
10 :  tensor(0.07827734, grad_fn=<AbsBackward>)
11 :  tensor(0.05372636, grad_fn=<AbsBackward>)
12 :  tensor(0.00887935, grad_fn=<AbsBackward>)
13 :  tensor(0.03301206, grad_fn=<AbsBackward>)
14 :  tensor(0.04595456, grad_fn=<AbsBackward>)
15 :  tensor(0.04316954, grad_fn=<AbsBackward>)
16 :  tensor(0.02975258, grad_fn=<AbsBackward>)
17 :  tensor(0.00406380, grad_fn=<AbsBackward>)
18 :  tensor(0.03266164, grad_fn=<AbsBackward>)
19 :  tensor(0.04991698, grad_fn=<AbsBackward>)
20 :  tensor(0.05560488, grad_fn=<AbsBackward>)
21 :  tensor(0.05205455, grad_fn=<AbsBackward

171 :  tensor(0.01224989, grad_fn=<AbsBackward>)
172 :  tensor(0.00120272, grad_fn=<AbsBackward>)
173 :  tensor(0.02054987, grad_fn=<AbsBackward>)
174 :  tensor(0.03121999, grad_fn=<AbsBackward>)
175 :  tensor(0.02903113, grad_fn=<AbsBackward>)
176 :  tensor(0.01758970, grad_fn=<AbsBackward>)
177 :  tensor(0.00602062, grad_fn=<AbsBackward>)
178 :  tensor(0.01159379, grad_fn=<AbsBackward>)
179 :  tensor(0.00999618, grad_fn=<AbsBackward>)
180 :  tensor(0.00218997, grad_fn=<AbsBackward>)
181 :  tensor(0.00319564, grad_fn=<AbsBackward>)
182 :  tensor(0.00678009, grad_fn=<AbsBackward>)
183 :  tensor(0.00675770, grad_fn=<AbsBackward>)
184 :  tensor(0.00649311, grad_fn=<AbsBackward>)
185 :  tensor(0.00775832, grad_fn=<AbsBackward>)
186 :  tensor(0.00039963, grad_fn=<AbsBackward>)
187 :  tensor(0.00206895, grad_fn=<AbsBackward>)
188 :  tensor(0.01261188, grad_fn=<AbsBackward>)
189 :  tensor(0.01030796, grad_fn=<AbsBackward>)
190 :  tensor(0.00033053, grad_fn=<AbsBackward>)
191 :  tensor(0.0185

339 :  tensor(0.02225360, grad_fn=<AbsBackward>)
340 :  tensor(0.00970665, grad_fn=<AbsBackward>)
341 :  tensor(0.01105108, grad_fn=<AbsBackward>)
342 :  tensor(0.02299466, grad_fn=<AbsBackward>)
343 :  tensor(0.02255547, grad_fn=<AbsBackward>)
344 :  tensor(0.00956033, grad_fn=<AbsBackward>)
345 :  tensor(0.00843559, grad_fn=<AbsBackward>)
346 :  tensor(0.01619023, grad_fn=<AbsBackward>)
347 :  tensor(0.01349891, grad_fn=<AbsBackward>)
348 :  tensor(0.00241601, grad_fn=<AbsBackward>)
349 :  tensor(0.02232255, grad_fn=<AbsBackward>)
350 :  tensor(0.02907500, grad_fn=<AbsBackward>)
351 :  tensor(0.02691822, grad_fn=<AbsBackward>)
352 :  tensor(0.01597906, grad_fn=<AbsBackward>)
353 :  tensor(0.00520446, grad_fn=<AbsBackward>)
354 :  tensor(0.01469582, grad_fn=<AbsBackward>)
355 :  tensor(0.01111408, grad_fn=<AbsBackward>)
356 :  tensor(0.00477308, grad_fn=<AbsBackward>)
357 :  tensor(0.00222554, grad_fn=<AbsBackward>)
358 :  tensor(0.00567749, grad_fn=<AbsBackward>)
359 :  tensor(0.0032

In [ ]:
print(trainfile[6])

In [20]:
torch.save(G1.state_dict(),"/home/hjj/Desktop/GCN-slab-61.pkl") 

In [13]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)


In [ ]:
print(testfile)

In [14]:

start=time.time()        
for path_ in testfile:
    try:
        GGG=get_cpx222_surface_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

942.9196565151215


In [ ]:
315.1618971824646

In [ ]:
GANs_Gmat(tomgStructure('/home/hjj/Desktop/GcN-sur/test/00837'))

In [15]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

0.0022623691279289025
5.133353554696426e-06


In [ ]:
print(np.sqrt(np.asarray(MSEset).mean()))

In [18]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_cpx222_surface_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [19]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

0.002229789486718152
4.9878858963132495e-06


In [ ]:

np.sqrt(0.01112732045941305)